In [1]:
# !pip install folktables

In [2]:
#info about Data
# https://arxiv.org/pdf/2108.04884.pdf

#ACSDataSource ,ACSEmployment, ACSPublicCoverage, ACSIncome

-> (using ACSEmployment) from the paper: predict whether an individual is employed, after filtering the ACS PUMS data
sample to only include individuals between the ages of 16 and 90  

In [1]:
# !pip install torchsummary

In [2]:
# !python --version

In [1]:
import pandas as pd
from collections import OrderedDict
from typing import OrderedDict
import numpy as np
import copy
import os
import torch
import torch.nn as nn
from sklearn import preprocessing
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from folktables import ACSDataSource, ACSEmployment,ACSIncome
import matplotlib.pyplot as plt
import torch.nn.functional as F
import tqdm
from sklearn.metrics import roc_curve
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
from torchvision import models
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, Subset, random_split
import torch.utils.data as data_utils
import pickle

In [2]:
# detail about the state: load_acs.py

In [3]:
from folktables import ACSDataSource, ACSEmployment,ACSIncome
data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
acs_data = data_source.get_data(states=["AL"], download=True)

In [4]:
# features_np, label_np, group_np = ACSEmployment.df_to_numpy(acs_data)
features, label, group = ACSEmployment.df_to_pandas(acs_data)

In [5]:
features

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,SEX,RAC1P
0,19.0,18.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0
1,18.0,18.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0
2,53.0,17.0,5.0,16.0,1.0,0.0,1.0,1.0,4.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0
3,28.0,19.0,5.0,16.0,2.0,0.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0
4,25.0,12.0,5.0,16.0,1.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47772,18.0,16.0,5.0,2.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0
47773,15.0,11.0,5.0,2.0,2.0,7.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0
47774,10.0,6.0,5.0,2.0,2.0,7.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0
47775,4.0,1.0,5.0,2.0,2.0,7.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,0.0,1.0,1.0


In [6]:
label

,ESR
0,False
1,False
2,False
3,False
4,False
...,...
47772,True
47773,False
47774,False
47775,False


In [7]:
merged_df=pd.concat([features, label], axis=1)
merged_df.head()

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,SEX,RAC1P,ESR
0,19.0,18.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,False
1,18.0,18.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,False
2,53.0,17.0,5.0,16.0,1.0,0.0,1.0,1.0,4.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,False
3,28.0,19.0,5.0,16.0,2.0,0.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False
4,25.0,12.0,5.0,16.0,1.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,False


In [8]:
len(merged_df)

47777

In [9]:
group.value_counts()

RAC1P
1        35551
2        10115
9          835
6          556
8          491
3          182
5           33
7           13
4            1
dtype: int64

In [10]:
merged_df["RAC1P"].value_counts()

# 1: White alone – 2: Black or African American alone

1.0    35551
2.0    10115
9.0      835
6.0      556
8.0      491
3.0      182
5.0       33
7.0       13
4.0        1
Name: RAC1P, dtype: int64

In [11]:
pd.unique(merged_df["RAC1P"])

array([1., 2., 8., 9., 6., 3., 5., 7., 4.])

In [12]:
#  for Black and white people
filtered_df= merged_df[["SEX","RAC1P","ESR"]][merged_df['RAC1P'].isin([1,2])]
print("len(filtered_df): ", len(filtered_df),"\n")
print(filtered_df["ESR"].value_counts(),"\n")
print(filtered_df["SEX"].value_counts(),"\n")
print(filtered_df["RAC1P"].value_counts(),"\n")
filtered_df.head() 

len(filtered_df):  45666 

False    26881
True     18785
Name: ESR, dtype: int64 

2.0    23737
1.0    21929
Name: SEX, dtype: int64 

1.0    35551
2.0    10115
Name: RAC1P, dtype: int64 



,SEX,RAC1P,ESR
0,2.0,1.0,False
1,2.0,2.0,False
2,1.0,1.0,False
3,1.0,1.0,False
4,2.0,1.0,False


In [13]:
# 1- Male 2- Female
# 1- White 2-Black

counts_frq = filtered_df.groupby(['SEX', 'RAC1P']).size()
counts_frq

SEX  RAC1P
1.0  1.0      17309
     2.0       4620
2.0  1.0      18242
     2.0       5495
dtype: int64

In [14]:
merged_df

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,SEX,RAC1P,ESR
0,19.0,18.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,False
1,18.0,18.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,False
2,53.0,17.0,5.0,16.0,1.0,0.0,1.0,1.0,4.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,False
3,28.0,19.0,5.0,16.0,2.0,0.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False
4,25.0,12.0,5.0,16.0,1.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47772,18.0,16.0,5.0,2.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,True
47773,15.0,11.0,5.0,2.0,2.0,7.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False
47774,10.0,6.0,5.0,2.0,2.0,7.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False
47775,4.0,1.0,5.0,2.0,2.0,7.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,0.0,1.0,1.0,False


In [15]:
# # filter the dataframe for each combination

#White Men
WM = merged_df.loc[(merged_df['SEX'] == 1) & (merged_df['RAC1P'] == 1)]
#Black Men
BM = merged_df.loc[(merged_df['SEX'] == 1) & (merged_df['RAC1P'] == 2)]
#white Women
WW = merged_df.loc[(merged_df['SEX'] == 2) & (merged_df['RAC1P'] == 1)]
#Black Women
BW = merged_df.loc[(merged_df['SEX'] == 2) & (merged_df['RAC1P'] == 2)]

In [16]:
WM

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,SEX,RAC1P,ESR
2,53.0,17.0,5.0,16.0,1.0,0.0,1.0,1.0,4.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,False
3,28.0,19.0,5.0,16.0,2.0,0.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False
7,38.0,12.0,5.0,16.0,1.0,0.0,1.0,1.0,4.0,4.0,1.0,1.0,2.0,2.0,1.0,1.0,False
8,41.0,16.0,5.0,17.0,2.0,0.0,1.0,1.0,4.0,4.0,1.0,2.0,2.0,2.0,1.0,1.0,True
14,21.0,19.0,5.0,17.0,2.0,0.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47768,76.0,13.0,3.0,0.0,1.0,0.0,1.0,1.0,4.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,False
47773,15.0,11.0,5.0,2.0,2.0,7.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False
47774,10.0,6.0,5.0,2.0,2.0,7.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False
47775,4.0,1.0,5.0,2.0,2.0,7.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,0.0,1.0,1.0,False


# Removing Sex and Gen from 4 Groups

In [17]:
WM.columns

Index(['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC',
       'NATIVITY', 'DEAR', 'DEYE', 'DREM', 'SEX', 'RAC1P', 'ESR'],
      dtype='object')

In [18]:
WM_updated=WM[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM','SEX', 'RAC1P', 'ESR']]
BM_updated=BM[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM','SEX', 'RAC1P', 'ESR']]
WW_updated=WW[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM','SEX', 'RAC1P', 'ESR']]
BW_updated=BW[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM','SEX', 'RAC1P', 'ESR']]

# Grouping

In [19]:
#Group-1
#Train set: White men+black men+Black women—- validation set: black men
WM_BM_BW_train = pd.concat([WM_updated,BM_updated,BW_updated])
BM_validation=BM_updated.copy()

#Group-2
#Train set: White women+black women+black men—- validation set: black women
WW_BW_BM_train=pd.concat([WW_updated,BW_updated,BM_updated])
BW_validation=BW_updated.copy()

#Group-3
#Train set: black men+white men+white women+validation set: white men
BM_WM_WW_train=pd.concat([BM_updated,WM_updated,WW_updated])
WM_validation=WM_updated.copy()

#Group-4
#Train set: white women+white men+black women, validation set: white women
WW_WM_BW_train=pd.concat([WW_updated,WM_updated,BW_updated])
WW_validation=WW_updated.copy()

In [20]:
print("BM" ,len(BM),"RAC: ", pd.unique(BM['RAC1P']),"SEX: ", pd.unique(BM['SEX']))
print("BW" ,len(BW),"RAC: ", pd.unique(BW['RAC1P']),"SEX: ", pd.unique(BW['SEX']))
print("WM" ,len(WM),"RAC: ", pd.unique(WM['RAC1P']),"SEX: ", pd.unique(WM['SEX']))
print("WW" ,len(WW),"RAC: ", pd.unique(WW['RAC1P']),"SEX: ", pd.unique(WW['SEX']))

BM 4620 RAC:  [2.] SEX:  [1.]
BW 5495 RAC:  [2.] SEX:  [2.]
WM 17309 RAC:  [1.] SEX:  [1.]
WW 18242 RAC:  [1.] SEX:  [2.]


In [21]:
data_to_use_train1= WM_BM_BW_train
data_to_use_val1= BM_validation

data_to_use_train2= WW_BW_BM_train
data_to_use_val2= BW_validation


data_to_use_train3= BM_WM_WW_train
data_to_use_val3= WM_validation


data_to_use_train4= WW_WM_BW_train
data_to_use_val4= WW_validation

In [22]:
len(WW_WM_BW_train)

41046

In [23]:

X_train1= data_to_use_train1[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM']]
y_train1= data_to_use_train1[['ESR']]

X_train2= data_to_use_train2[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM']]
y_train2= data_to_use_train2[['ESR']]

X_train3= data_to_use_train3[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM']]
y_train3= data_to_use_train3[['ESR']]

X_train4= data_to_use_train4[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM']]
y_train4= data_to_use_train4[['ESR']]

# X_train = torch.from_numpy(X_train.to_numpy()).float()
# y_train = torch.squeeze(torch.from_numpy(y_train.to_numpy()).float())


In [24]:
X_train2

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM
0,19.0,18.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0
4,25.0,12.0,5.0,16.0,1.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,1.0
5,30.0,16.0,5.0,16.0,1.0,0.0,1.0,1.0,4.0,4.0,1.0,2.0,2.0,1.0
6,66.0,19.0,2.0,16.0,1.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,1.0
9,18.0,18.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,2.0,1.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47709,43.0,19.0,1.0,0.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0
47711,17.0,19.0,5.0,2.0,2.0,1.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0
47712,29.0,19.0,1.0,0.0,2.0,0.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0
47752,6.0,3.0,5.0,7.0,2.0,7.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0


In [25]:
# With Sex and Race
X_train2_1= data_to_use_train2[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM','SEX', 'RAC1P']]
y_train2_1= data_to_use_train2[['ESR']]

In [26]:
X_train2_1

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,SEX,RAC1P
0,19.0,18.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0
4,25.0,12.0,5.0,16.0,1.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0
5,30.0,16.0,5.0,16.0,1.0,0.0,1.0,1.0,4.0,4.0,1.0,2.0,2.0,1.0,2.0,1.0
6,66.0,19.0,2.0,16.0,1.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0
9,18.0,18.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47709,43.0,19.0,1.0,0.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0
47711,17.0,19.0,5.0,2.0,2.0,1.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0
47712,29.0,19.0,1.0,0.0,2.0,0.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0
47752,6.0,3.0,5.0,7.0,2.0,7.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0


In [27]:
X_valid1 = data_to_use_val1[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM']]
y_valid1 = data_to_use_val1[['ESR']]

X_valid2 = data_to_use_val2[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM']]
y_valid2 = data_to_use_val2[['ESR']]

X_valid3 = data_to_use_val3[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM']]
y_valid3 = data_to_use_val3[['ESR']]

X_valid4 = data_to_use_val4[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM']]
y_valid4 = data_to_use_val4[['ESR']]

# X_valid = torch.from_numpy(X_valid.to_numpy()).float()
# y_valid = torch.squeeze(torch.from_numpy(y_valid.to_numpy()).float())

# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

In [28]:
# with SEX and RACE

X_valid2_1 = data_to_use_val2[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM','SEX', 'RAC1P']]
y_valid2_1 = data_to_use_val2[['ESR']]

In [29]:
print("TYPE X: ",type(X_train1)," Type Y", type(y_train1),"\n")
print("len(X_train),len(y_train), len(X_val),len(y_val): ",len(X_train1),len(y_train1), len(X_valid1),len(y_valid1))
print("X_train.shape, y_train.shape ",X_train1.shape, y_train1.shape,"\n")
print("X_valid.shape, y_valid.shape ",X_valid1.shape, y_valid1.shape,"\n")

X_train1

TYPE X:  <class 'pandas.core.frame.DataFrame'>  Type Y <class 'pandas.core.frame.DataFrame'> 

len(X_train),len(y_train), len(X_val),len(y_val):  27424 27424 4620 4620
X_train.shape, y_train.shape  (27424, 14) (27424, 1) 

X_valid.shape, y_valid.shape  (4620, 14) (4620, 1) 



,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM
2,53.0,17.0,5.0,16.0,1.0,0.0,1.0,1.0,4.0,2.0,1.0,2.0,2.0,1.0
3,28.0,19.0,5.0,16.0,2.0,0.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0
7,38.0,12.0,5.0,16.0,1.0,0.0,1.0,1.0,4.0,4.0,1.0,1.0,2.0,2.0
8,41.0,16.0,5.0,17.0,2.0,0.0,1.0,1.0,4.0,4.0,1.0,2.0,2.0,2.0
14,21.0,19.0,5.0,17.0,2.0,0.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47710,28.0,19.0,1.0,1.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0
47750,56.0,16.0,5.0,0.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0
47751,25.0,21.0,5.0,2.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0
47764,81.0,14.0,1.0,1.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0


In [30]:
y_train1

,ESR
2,False
3,False
7,False
8,True
14,False
...,...
47710,True
47750,False
47751,True
47764,False


In [31]:
X_valid1 = preprocessing.normalize(X_valid1)
X_train1 = preprocessing.normalize(X_train1)

X_valid2 = preprocessing.normalize(X_valid2)
X_train2 = preprocessing.normalize(X_train2)

X_valid3 = preprocessing.normalize(X_valid3)
X_train3 = preprocessing.normalize(X_train3)

X_valid4 = preprocessing.normalize(X_valid4)
X_train4 = preprocessing.normalize(X_train4)


In [32]:
X_valid2_1 = preprocessing.normalize(X_valid2_1)
X_train2_1 = preprocessing.normalize(X_train2_1)

In [33]:
encoder = LabelEncoder()


In [34]:
# Binary encoding of labels
encoder.fit(y_train1)

C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\preprocessing\_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LabelEncoder()

In [35]:
y_train1 = encoder.transform(y_train1)
y_valid1 = encoder.transform(y_valid1)

y_train2 = encoder.transform(y_train2)
y_valid2 = encoder.transform(y_valid2)

y_train3 = encoder.transform(y_train3)
y_valid3 = encoder.transform(y_valid3)

y_train4 = encoder.transform(y_train4)
y_valid4 = encoder.transform(y_valid4)


C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\preprocessing\_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\preprocessing\_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\preprocessing\_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
C:\Users\Admin\AppData\Lo

In [36]:
# With Sex and Race

y_train2_1 = encoder.transform(y_train2_1)
y_valid2_1 = encoder.transform(y_valid2_1)


C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\preprocessing\_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\preprocessing\_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [37]:
# Convert to 2D PyTorch tensors
X_train1= torch.tensor(X_train1, dtype=torch.float32)
X_valid1= torch.tensor(X_valid1, dtype=torch.float32)
y_train1 = torch.tensor(y_train1, dtype=torch.float32).reshape(-1, 1)
y_valid1 = torch.tensor(y_valid1, dtype=torch.float32).reshape(-1, 1)


X_train2 = torch.tensor(X_train2, dtype=torch.float32)
X_valid2 = torch.tensor(X_valid2, dtype=torch.float32)
y_train2 = torch.tensor(y_train2, dtype=torch.float32).reshape(-1, 1)
y_valid2 = torch.tensor(y_valid2, dtype=torch.float32).reshape(-1, 1)


X_train3= torch.tensor(X_train3, dtype=torch.float32)
X_valid3= torch.tensor(X_valid3, dtype=torch.float32)
y_train3 = torch.tensor(y_train3, dtype=torch.float32).reshape(-1, 1)
y_valid3 = torch.tensor(y_valid3, dtype=torch.float32).reshape(-1, 1)



X_train4= torch.tensor(X_train4, dtype=torch.float32)
X_valid4= torch.tensor(X_valid4, dtype=torch.float32)
y_train4 = torch.tensor(y_train4, dtype=torch.float32).reshape(-1, 1)
y_valid4 = torch.tensor(y_valid4, dtype=torch.float32).reshape(-1, 1)

In [38]:
#with Sex and Race

X_train2_1 = torch.tensor(X_train2_1, dtype=torch.float32)
X_valid2_1 = torch.tensor(X_valid2_1, dtype=torch.float32)
y_train2_1 = torch.tensor(y_train2_1, dtype=torch.float32).reshape(-1, 1)
y_valid2_1 = torch.tensor(y_valid2_1, dtype=torch.float32).reshape(-1, 1)



In [39]:
X_train1,y_train1

(tensor([[0.9073, 0.2910, 0.0856,  ..., 0.0342, 0.0342, 0.0171],
         [0.7353, 0.4990, 0.1313,  ..., 0.0525, 0.0525, 0.0525],
         [0.8686, 0.2743, 0.1143,  ..., 0.0229, 0.0457, 0.0457],
         ...,
         [0.7434, 0.6244, 0.1487,  ..., 0.0595, 0.0595, 0.0595],
         [0.9826, 0.1698, 0.0121,  ..., 0.0243, 0.0243, 0.0243],
         [0.9561, 0.2684, 0.0503,  ..., 0.0335, 0.0335, 0.0335]]),
 tensor([[0.],
         [0.],
         [0.],
         ...,
         [1.],
         [0.],
         [0.]]))

In [40]:
y_train1,y_valid1

(tensor([[0.],
         [0.],
         [0.],
         ...,
         [1.],
         [0.],
         [0.]]),
 tensor([[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]))

In [41]:
X_train1.shape, y_train1.shape

(torch.Size([27424, 14]), torch.Size([27424, 1]))

# data loader-torch

In [42]:
BATCH=1000

In [43]:

# Passing to DataLoader


train1 = data_utils.TensorDataset(X_train1, y_train1)
valid1 = data_utils.TensorDataset(X_valid1, y_valid1)

train2 = data_utils.TensorDataset(X_train2, y_train2)
valid2 = data_utils.TensorDataset(X_valid2, y_valid2)

train3 = data_utils.TensorDataset(X_train3, y_train3)
valid3 = data_utils.TensorDataset(X_valid3, y_valid3)

train4 = data_utils.TensorDataset(X_train4, y_train4)
valid4 = data_utils.TensorDataset(X_valid4, y_valid4)

trainloader1 = data_utils.DataLoader(train1, batch_size=BATCH, shuffle=True)
trainloader2 = data_utils.DataLoader(train2, batch_size=BATCH, shuffle=True)
trainloader3 = data_utils.DataLoader(train3, batch_size=BATCH, shuffle=True)
trainloader4 = data_utils.DataLoader(train4, batch_size=BATCH, shuffle=True)

valloader1 = data_utils.DataLoader(valid1, batch_size=BATCH)
valloader2 = data_utils.DataLoader(valid2, batch_size=BATCH)
valloader3 = data_utils.DataLoader(valid3, batch_size=BATCH)
valloader4 = data_utils.DataLoader(valid4, batch_size=BATCH)




In [44]:
#with sex and race


train2_1 = data_utils.TensorDataset(X_train2_1, y_train2_1)
valid2_1 = data_utils.TensorDataset(X_valid2_1, y_valid2_1)

trainloader2_1 = data_utils.DataLoader(train2_1, batch_size=BATCH, shuffle=True)
valloader2_1 = data_utils.DataLoader(valid2_1, batch_size=BATCH)


In [45]:
# train_data_all_client=[trainloader1,trainloader2,trainloader3,trainloader4]
# val_data_all_client=[valloader1,valloader2,valloader3,valloader4]

In [46]:
# with open("D:/Download/pythonProject/HiWi/pFedLA_Folktable/train_1_client.pkl", "wb") as f:
#     pickle.dump(train_data_all_client, f)
    
# with open("D:/Download/pythonProject/HiWi/pFedLA_Folktable/valid_1_client.pkl", "wb") as f:
#     pickle.dump(val_data_all_client, f)

In [62]:
class DeepNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 14 : input shape
        self.layer1 = nn.Linear(14, 512)
        self.act1 = nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.5)
        self.layer2 = nn.Linear(512, 256)
        self.act2 = nn.ReLU()
        self.layer3 = nn.Linear(256, 60)
        self.act3 = nn.ReLU()
        self.output = nn.Linear(60, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.layer1(x))
        x = self.dropout1(x)
        x = self.act2(self.layer2(x))
        x = self.act3(self.layer3(x))
        x = self.sigmoid(self.output(x))
        return x    

In [47]:
#With Sex and RACE

class DeepNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 14 : input shape
        self.layer1 = nn.Linear(16, 512)
        self.act1 = nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.5)
        self.layer2 = nn.Linear(512, 256)
        self.act2 = nn.ReLU()
        self.layer3 = nn.Linear(256, 60)
        self.act3 = nn.ReLU()
        self.output = nn.Linear(60, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.layer1(x))
        x = self.dropout1(x)
        x = self.act2(self.layer2(x))
        x = self.act3(self.layer3(x))
        x = self.sigmoid(self.output(x))
        return x    

# training with Loaders

In [48]:
# data_to_use_train1= WM_BM_BW_train
# data_to_use_val1= BM_validation

# data_to_use_train2= WW_BW_BM_train
# data_to_use_val2= BW_validation


# data_to_use_train3= BM_WM_WW_train
# data_to_use_val3= WM_validation


# data_to_use_train4= WW_WM_BW_train
# data_to_use_val4= WW_validation

In [49]:
model = DeepNet()

# Define loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # You can adjust the learning rate

# Move the model to the appropriate device (CPU or GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

DeepNet(
  (layer1): Linear(in_features=16, out_features=512, bias=True)
  (act1): ReLU()
  (dropout1): Dropout(p=0.5, inplace=False)
  (layer2): Linear(in_features=512, out_features=256, bias=True)
  (act2): ReLU()
  (layer3): Linear(in_features=256, out_features=60, bias=True)
  (act3): ReLU()
  (output): Linear(in_features=60, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [50]:
final__train_lodear=trainloader2_1
final_val_lodear=valloader2_1
vaL_acc_list=[]

# Training loop
num_epochs = 20  # Adjust the number of epochs as needed
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    for batch_idx, (inputs, targets) in enumerate(final__train_lodear):
        inputs, targets = inputs.to(device), targets.to(device)
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        
        # Compute the loss
        loss = criterion(outputs, targets)
        
        # Backpropagation
        loss.backward()
        
        # Update the weights
        optimizer.step()
        
        if (batch_idx + 1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(final__train_lodear)}], Loss: {loss.item():.4f}")

            # Validation loop
            model.eval()  # Set the model to evaluation mode
            with torch.no_grad():
                correct = 0
                total = 0
                val_loss=0
                for inputs, targets in final_val_lodear:
                    inputs, targets = inputs.to(device), targets.to(device)
                    outputs = model(inputs)
                    
                    val_loss += criterion(outputs, targets)
                    # You can define your own accuracy calculation here
                    # For binary classification, you can use something like this:
                    predicted = outputs > 0.5
                    total += targets.size(0)
                    correct += (predicted == targets).sum().item()
                
                val_loss = val_loss / len(final_val_lodear)
                val_acc = correct / total
                vaL_acc_list.append(val_acc)
#             print(f"Validation Accuracy: {val_acc:.2%}", f"Val Loss: {val_loss}")

print("\n\n Val acc",sorted(vaL_acc_list)[-1])

Epoch [1/20], Batch [10/29], Loss: 0.6495
Epoch [1/20], Batch [20/29], Loss: 0.6019
Epoch [2/20], Batch [10/29], Loss: 0.5522
Epoch [2/20], Batch [20/29], Loss: 0.4970
Epoch [3/20], Batch [10/29], Loss: 0.4696
Epoch [3/20], Batch [20/29], Loss: 0.4771
Epoch [4/20], Batch [10/29], Loss: 0.4925
Epoch [4/20], Batch [20/29], Loss: 0.4710
Epoch [5/20], Batch [10/29], Loss: 0.4710
Epoch [5/20], Batch [20/29], Loss: 0.4452
Epoch [6/20], Batch [10/29], Loss: 0.4862
Epoch [6/20], Batch [20/29], Loss: 0.4440
Epoch [7/20], Batch [10/29], Loss: 0.4907
Epoch [7/20], Batch [20/29], Loss: 0.4305
Epoch [8/20], Batch [10/29], Loss: 0.4611
Epoch [8/20], Batch [20/29], Loss: 0.4424
Epoch [9/20], Batch [10/29], Loss: 0.4678
Epoch [9/20], Batch [20/29], Loss: 0.4516
Epoch [10/20], Batch [10/29], Loss: 0.4782
Epoch [10/20], Batch [20/29], Loss: 0.4265
Epoch [11/20], Batch [10/29], Loss: 0.4666
Epoch [11/20], Batch [20/29], Loss: 0.4388
Epoch [12/20], Batch [10/29], Loss: 0.4513
Epoch [12/20], Batch [20/29],

In [51]:
name_of_model='0_WM_BM_BW_1.pth'

In [52]:
torch.save(model.state_dict(), name_of_model)

# Testing

In [53]:
name_of_model="0_WM_BM_BW_1.pth"

In [54]:
class DeepNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 14 : input shape
        self.layer1 = nn.Linear(16, 512)
        self.act1 = nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.5)
        self.layer2 = nn.Linear(512, 256)
        self.act2 = nn.ReLU()
        self.layer3 = nn.Linear(256, 60)
        self.act3 = nn.ReLU()
        self.output = nn.Linear(60, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.layer1(x))
        x = self.dropout1(x)
        x = self.act2(self.layer2(x))
        x = self.act3(self.layer3(x))
        x = self.sigmoid(self.output(x))
        return x    

In [55]:
model = DeepNet()
# model = Deep_wide_Net()
model.load_state_dict(torch.load(name_of_model))
model.eval()

DeepNet(
  (layer1): Linear(in_features=16, out_features=512, bias=True)
  (act1): ReLU()
  (dropout1): Dropout(p=0.5, inplace=False)
  (layer2): Linear(in_features=512, out_features=256, bias=True)
  (act2): ReLU()
  (layer3): Linear(in_features=256, out_features=60, bias=True)
  (act3): ReLU()
  (output): Linear(in_features=60, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [56]:
data_source_test = ACSDataSource(survey_year='2017', horizon='1-Year', survey='person')
acs_data_test = data_source_test.get_data(states=["AL"], download=True)
features_test, label_test, group_test = ACSEmployment.df_to_pandas(acs_data_test)
print(features_test.head(),"\n\n Lable: \n ", label_test.head())

   AGEP  SCHL  MAR  RELP  DIS  ESP  CIT  MIG  MIL  ANC  NATIVITY  DEAR  DEYE  \
0  73.0  10.0  4.0   0.0  1.0  0.0  1.0  1.0  4.0  1.0       1.0   2.0   2.0   
1  31.0  21.0  5.0   0.0  2.0  0.0  1.0  1.0  4.0  2.0       1.0   2.0   2.0   
2  41.0  17.0  1.0   0.0  2.0  0.0  1.0  1.0  4.0  1.0       1.0   2.0   2.0   
3  48.0  16.0  1.0   1.0  2.0  0.0  1.0  1.0  4.0  1.0       1.0   2.0   2.0   
4  16.0  13.0  5.0   2.0  2.0  2.0  1.0  1.0  0.0  1.0       1.0   2.0   2.0   

   DREM  SEX  RAC1P  
0   1.0  1.0    2.0  
1   2.0  2.0    1.0  
2   2.0  1.0    1.0  
3   2.0  2.0    1.0  
4   2.0  1.0    1.0   

 Lable: 
       ESR
0  False
1   True
2   True
3  False
4  False


In [57]:
len(features_test)

47645

In [58]:
merged_df_test=pd.concat([features_test, label_test], axis=1)
merged_df_test

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,SEX,RAC1P,ESR
0,73.0,10.0,4.0,0.0,1.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,False
1,31.0,21.0,5.0,0.0,2.0,0.0,1.0,1.0,4.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,True
2,41.0,17.0,1.0,0.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,True
3,48.0,16.0,1.0,1.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,False
4,16.0,13.0,5.0,2.0,2.0,2.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47640,20.0,18.0,5.0,17.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False
47641,18.0,16.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,True
47642,25.0,17.0,1.0,16.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,9.0,False
47643,24.0,17.0,5.0,16.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False


In [59]:
# # filter the dataframe for each combination

#White Men
WM_test = merged_df_test.loc[(merged_df_test['SEX'] == 1) & (merged_df_test['RAC1P'] == 1)]
#Black Men
BM_test = merged_df_test.loc[(merged_df_test['SEX'] == 1) & (merged_df_test['RAC1P'] == 2)]
#white Women
WW_test = merged_df_test.loc[(merged_df_test['SEX'] == 2) & (merged_df_test['RAC1P'] == 1)]
#Black Women
BW_test = merged_df_test.loc[(merged_df_test['SEX'] == 2) & (merged_df_test['RAC1P'] == 2)]

In [60]:
WM_BM_BW_test= pd.concat([WM_test,BM_test,BW_test])

WW_BW_BM_test= pd.concat([WW_test,BW_test,BM_test])

BM_WM_WW_test= pd.concat([WM_test,WM_test,WW_test])

WW_WM_BW_test= pd.concat([WW_test,WM_test,BW_test])

In [61]:
WM_BM_BW_test

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,SEX,RAC1P,ESR
2,41.0,17.0,1.0,0.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,True
4,16.0,13.0,5.0,2.0,2.0,2.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False
10,32.0,16.0,1.0,0.0,2.0,0.0,5.0,3.0,4.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,True
18,80.0,16.0,1.0,1.0,2.0,0.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,False
19,32.0,22.0,5.0,0.0,2.0,0.0,1.0,1.0,4.0,4.0,1.0,2.0,2.0,2.0,1.0,1.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47605,24.0,19.0,5.0,17.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,False
47623,20.0,16.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,True
47628,17.0,16.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,True
47635,19.0,19.0,5.0,17.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,False


In [68]:
counts_frq_test = merged_df_test.groupby(['SEX', 'RAC1P','ESR']).size()
counts_frq_test

SEX  RAC1P  ESR  
1.0  1.0    False     8880
            True      7903
     2.0    False     3352
            True      1636
     3.0    False       67
            True        43
     4.0    True         1
     5.0    False       21
            True         7
     6.0    False      144
            True       146
     7.0    False        4
            True         4
     8.0    False      132
            True       121
     9.0    False      289
            True       106
2.0  1.0    False    11326
            True      6718
     2.0    False     3483
            True      2170
     3.0    False       59
            True        38
     4.0    False        1
     5.0    False       12
            True         7
     6.0    False      196
            True       131
     7.0    False        6
            True         1
     8.0    False      152
            True        63
     9.0    False      306
            True       120
dtype: int64

In [63]:
merged_df_test  

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,SEX,RAC1P,ESR
0,73.0,10.0,4.0,0.0,1.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,False
1,31.0,21.0,5.0,0.0,2.0,0.0,1.0,1.0,4.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,True
2,41.0,17.0,1.0,0.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,True
3,48.0,16.0,1.0,1.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,False
4,16.0,13.0,5.0,2.0,2.0,2.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47640,20.0,18.0,5.0,17.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False
47641,18.0,16.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,True
47642,25.0,17.0,1.0,16.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,9.0,False
47643,24.0,17.0,5.0,16.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False


In [64]:
# X_test= WM_BM_BW_test[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM']]
# y_test= WM_BM_BW_test[['ESR']]


X_test= WM_BM_BW_test[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM','SEX','RAC1P']]
y_test= WM_BM_BW_test[['ESR']]


# X_test= merged_df_test[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM']]
# y_test= merged_df_test[['ESR']]


X_test = preprocessing.normalize(X_test)
encoder = LabelEncoder()
encoder.fit(y_test)
y_test = encoder.transform(y_test)

len(X_test)

C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\preprocessing\_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\preprocessing\_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


27424

In [65]:
X_test= torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)
print(X_test)
print(y_test)


tensor([[0.9147, 0.3793, 0.0223,  ..., 0.0446, 0.0223, 0.0223],
        [0.7303, 0.5934, 0.2282,  ..., 0.0913, 0.0456, 0.0456],
        [0.8696, 0.4348, 0.0272,  ..., 0.0544, 0.0272, 0.0272],
        ...,
        [0.5623, 0.5292, 0.1654,  ..., 0.0662, 0.0662, 0.0662],
        [0.5782, 0.5782, 0.1521,  ..., 0.0609, 0.0609, 0.0609],
        [0.6722, 0.5378, 0.1344,  ..., 0.0538, 0.0538, 0.0538]])
tensor([[1.],
        [0.],
        [1.],
        ...,
        [1.],
        [0.],
        [0.]])


In [66]:
BATCH=1000

test_data = data_utils.TensorDataset(X_test, y_test)

test_loader = data_utils.DataLoader(test_data, batch_size=BATCH, shuffle=True)

In [67]:
# Testing loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()  # Set the model to evaluation mode

with torch.no_grad():
    correct = 0
    total = 0
    loss=0
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)

        loss += criterion(outputs, targets)
        
        predicted = outputs > 0.5
                
        total += targets.size(0)
        correct += (predicted == targets).sum().item()
        
    print("Correct: ",correct,"Total: ", total)
    
    average_loss = loss / len(test_loader)
    accuracy = correct / total
    
    print(f"Testing Accuracy: {accuracy:.2%}")
    print(f"Testing Loss: " ,average_loss)

Correct:  22399 Total:  27424
Testing Accuracy: 81.68%
Testing Loss:  tensor(0.4084)


# Testing FL model

In [25]:
class DeepNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(14, 512)
        self.act1 = nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.5)
        self.layer2 = nn.Linear(512, 256)
        self.act2 = nn.ReLU()
        self.layer3 = nn.Linear(256, 60)
        self.act3 = nn.ReLU()
        self.output = nn.Linear(60, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.layer1(x))
        x = self.dropout1(x)
        x = self.act2(self.layer2(x))
        x = self.act3(self.layer3(x))
        x = self.sigmoid(self.output(x))
        return x    

In [34]:
# total client 4
name_of_model="../temp/my_model/global_model.pt"
model = DeepNet()
model.load_state_dict(torch.load(name_of_model))

<All keys matched successfully>

In [36]:
model.eval()  # Set the model to evaluation mode

with torch.no_grad():
    correct = 0
    total = 0
    loss=0
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)

        loss += criterion(outputs, targets)
        
        predicted = outputs > 0.5
                
        total += targets.size(0)
        correct += (predicted == targets).sum().item()
        
    print("Correct: ",correct,"Total: ", total)
    
    average_loss = loss / len(test_loader)
    accuracy = correct / total
    
    print(f"Testing Accuracy: {accuracy:.2%}")
    print(f"Testing Loss: " ,average_loss)

Correct:  22098 Total:  27424
Testing Accuracy: 80.58%
Testing Loss:  tensor(0.4146)


In [37]:
# Testing loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()  # Set the model to evaluation mode

with torch.no_grad():
    correct = 0
    total = 0
    loss=0
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)

        loss += criterion(outputs, targets)
        
        predicted = outputs > 0.5
                
        total += targets.size(0)
        correct += (predicted == targets).sum().item()
        
    print("Correct: ",correct,"Total: ", total)
    
    average_loss = loss / len(test_loader)
    accuracy = correct / total
    
    print(f"Testing Accuracy: {accuracy:.2%}")
    print(f"Testing Loss: " ,average_loss)

Correct:  15715 Total:  27424
Testing Accuracy: 57.30%
Testing Loss:  tensor(0.6912)
